In [ ]:
from pathlib import Path  # Path manipulation for file handling
from skimage.io import imread, util  # Image reading functionality
import numpy as np  # Numerical operations
import cv2  # Computer vision library
import matplotlib.pyplot as plt  # Plotting library
import os  # Operating system interaction
import pandas as pd  # Data manipulation and analysis
import time as t # Time-related functionality
import seaborn as sns
import math


In [ ]:
# Define the folder path where the CSV file is located
directory = r"E:\Data_to_analyze\DIC\frpr-8\R2"
directory_path = Path(directory)
# Define the name for the file   
filename = '3.12_AX7163&N2002.tif'

# Create the full path for the image
directory_filename = os.path.join(directory_path, filename)

# Read the image using skimage.io.imread
images = imread(directory_filename)
images.shape

In [ ]:
strain_name = input('write the strain name')
field_of_view = '_'+input('write the field of view')

strain_field_of_view = images[:,:,:950] # Slice the 'images' array to keep only the portion with worms of the strain slected
plt.imshow(strain_field_of_view[0])

In [ ]:
# Import functions from the custom module binding_box_finder
from binding_box_finder_v17 import blob_maker_DIC, binding_box_find_and_display

# Apply blob_maker function to create a binary image with blobs
bit8 = blob_maker_DIC(strain_field_of_view, 10)
# Use binding_box_find_and_display to find bounding boxes and display them on the original image
# This function returns coordinates and an image with bounding boxes
coord_list, image_boxes = binding_box_find_and_display(bit8, strain_field_of_view, 15)

In [ ]:
# List of indices to be removed from coord_list
indices_to_remove = [5,10,13,14,16]

# Use list comprehension to create a new coord_list excluding elements with specified indices
# Only include items in the new list if their index is not in indices_to_remove
coord_list = [item for index, item in enumerate(coord_list) if index not in indices_to_remove]

# Display the updated coord_list
coord_list

In [ ]:
# List of indices to keep in coord_list
indices_to_keep = [1, 4,8,7,6,11]

# Use list comprehension to create a new coord_list with elements from specified indices
# Only include items in the new list if their index is in indices_to_keep
coord_list = [item for index, item in enumerate(coord_list) if index in indices_to_keep]

# Display the updated coord_list
coord_list

In [ ]:
#append a coordinate
coord_to_append = [[100, 366, 50, 50]] #x, y, w, h top-left corner
for c in coord_to_append:
    coord_list.append(c)
coord_list

In [ ]:
# Import the draw_boxes_from_coordinates function from the custom module binding_box_finder
from binding_box_finder_v17 import draw_boxes_from_coordinates as dbfc

# Specify the filename for the image with drawn bounding boxes
filename_boxes = os.path.join(directory_path, 'boxes_' + strain_name +field_of_view +'.jpeg')
starting_n_worm = int(input('What is the starting number?'))

# Use the draw_boxes_from_coordinates function to draw bounding boxes on the first image in images_gcamp
# The function takes coord_list, the image, and the filename as parameters
_ = dbfc(coord_list, strain_field_of_view[0], filename_boxes, starting_n_worm)

In [ ]:

FPS = 0.2 # Define the frame rate (frames per second)

start_time = t.time()  # Record the start time for execution measurement
intensities_dict = {}  # Initialize an empty dictionary to store intensity values
intensities_dict_list = []  # Reinitialize the list for storing results

# Iterate over each worm's coordinates in coord_list, starting from 'starting_n_worm'
for n_worm, coord in enumerate(coord_list, start=starting_n_worm):
    x, y, w, h = coord
    print(n_worm)
    
    # Iterate through time points in the strain field of view (excluding the last frame)
    for time_point in range(len(strain_field_of_view)-1):
        frame_1 = strain_field_of_view[time_point]
        frame_2 = strain_field_of_view[time_point + 1]

        # Extract the worm region from both frames using the bounding box
        worm_1 = frame_1[y:(y+h), x:(x+w)]
        worm_2 = frame_2[y:(y+h), x:(x+w)]

         # Compute the intensity difference between consecutive frames using image subtraction
        intensity_imsubtraction = np.mean(cv2.subtract(worm_2, worm_1))
        
                # Store the results in a dictionary with worm index, time, and intensity value
        intensities_dict = {
            'n_worm': n_worm, 
            'time(s)': (time_point + 1) / FPS,  # Convert frame index to time in seconds
            'intensity': intensity_imsubtraction
        }
        # Append the dictionary to the list for later analysis
        intensities_dict_list.append(intensities_dict)

end_time = t.time()
ex_time = round(end_time - start_time,2)

ex_time # Output the execution time

        

In [ ]:
 # Convert the list of dictionaries into a pandas DataFrame and save it
intensities_df = pd.DataFrame(intensities_dict_list)
df_name = strain_name+field_of_view + '_intensities_df.csv'
intensities_df.to_csv(os.path.join(directory_path, df_name), index=False)
intensities_df

In [ ]:
from bout_detection_v7 import bout_detection_v2

# Define the number of frames corresponding to the minimum sleep bout length
minimum_sleep_bout_frame = int(FPS * 120) # Convert 120 seconds into frame count
thresh = 0.2# Define the speed threshold for detecting bouts

# Apply the bout_detection function to each worm's intensity data
# Group by 'n_worm' and apply the function to each group
result_df = intensities_df.groupby('n_worm').apply(lambda group: pd.Series(bout_detection_v2(group['intensity'], timethresh=minimum_sleep_bout_frame, speedthresh=thresh, window=40)))

# Reset index for a cleaner DataFrame
result_df = result_df.reset_index()


# Rename the columns if needed
result_df.columns = ['n_worm', 'q', 'normspeed', 'true_start', 'true_end']

# Display the result
result_df

In [ ]:
bout_data = []

# Calculate the number of unique worms and determine subplot layout
num_unique_worms = math.ceil(len(result_df['n_worm'].unique())/2)
# Create subplots with an appropriate figure size
fig, axs = plt.subplots(nrows=num_unique_worms, ncols=2, figsize=(20, 6 * num_unique_worms))


# Iterate through each worm's data
for worm, data in result_df.groupby('n_worm'):
    starts_list = data['true_start'].iloc[0]
    ends_list = data['true_end'].iloc[0]
    bouts = []
     # Calculate bout lengths in seconds
    for start, end in zip(starts_list, ends_list):
            bouts.append((end-start)/FPS)  # Convert frame count to time in seconds
    # Store the bout data in a dictionary
    bout_data.append({'worm' : (data['n_worm'].iloc[0]), 'bout_length': bouts})

# Flatten the subplot array for easy indexing
axs_flat = axs.flatten()

# Initialize row and col indices
row = 0
col = 0

# Plot 'normspeed' and 'q' for each worm
for worm, data in result_df.groupby('n_worm'):
    axs_flat[row * 2 + col].plot(data['normspeed'].iloc[0], label=f'Worm {worm} - normspeed')
    axs_flat[row * 2 + col].plot(data['q'].iloc[0], label=f'Worm {worm} - q')
    
    # Add a horizontal line at y=0.2 for each subplot
    axs_flat[row * 2 + col].axhline(y=thresh, color='red', linestyle='--')
    
    axs_flat[row * 2 + col].set_xlabel('Time')
    axs_flat[row * 2 + col].set_ylabel('Normalized Speed')  # Adjust the label based on your data
    axs_flat[row * 2 + col].set_title(f'Normalized Speed and q for Worm {worm}')
    
    # Update row and col indices
    if col < 1:
        col = col + 1
    elif col == 1:
        col = 0
        row = row + 1
    
    



# Show the plot
plt.show()
plot_name = f'{strain_name}{field_of_view}_sleep_plot.jpeg'
fig.savefig(os.path.join(directory_path, plot_name))

In [ ]:
#**resuslts in SECONDS**
bout_data_df = pd.DataFrame(bout_data)
bout_data_df

In [ ]:
# Calculate the total movie duration in seconds
movie_duration_sec = int((len(strain_field_of_view)-1)/FPS)

# Compute sleep metrics for each worm
average_bout_lengths = round(bout_data_df['bout_length'].apply(lambda x: (sum(x) / len(x))/60 if sum(x) != 0 else 0), 1) #bout length in minutes 
sleep_fraction = round(bout_data_df['bout_length'].apply(sum) / movie_duration_sec, 2) 
bout_frequency = bout_data_df['bout_length'].apply(len) / ((movie_duration_sec)/3600) 

# Create a DataFrame to store sleep analysis results
result_data = {
    'worm': (bout_data_df['worm']),
    'average_bout_length': average_bout_lengths.values,
    'sleep_fraction': sleep_fraction.values,
    'bout_frequency': bout_frequency.values,
    'strain': strain_name,
    'replicate': input('insert the nuber of replicate') 
}
sleep_data_df = pd.DataFrame(result_data)


sleep_data_name = 'sleep_data_' + strain_name+field_of_view + '.csv'
sleep_data_df.to_csv(os.path.join(directory_path, sleep_data_name), index=False) 

sleep_data_df